In [1]:
# Instalar la librería PRAW
pip install praw



   ------------- -------------------------- 1/3 [prawcore]
   -------------------------- ------------- 2/3 [praw]
   -------------------------- ------------- 2/3 [praw]
   -------------------------- ------------- 2/3 [praw]
   -------------------------- ------------- 2/3 [praw]
   -------------------------- ------------- 2/3 [praw]
   -------------------------- ------------- 2/3 [praw]
   ---------------------------------------- 3/3 [praw]

Note: you may need to restart the kernel to use updated packages.


In [5]:
# PART 2: COLLECT DATA AND STORAGE
# Collect posts from SubReddits

In [24]:
# Librerías
import csv # permite escribir archivos csv
from pathlib import Path # servirá para crear una carpeta en la cual guardar los archivos csv
import praw

# Credenciales
CLIENT_ID = "w2JQaemojvp2ZhiLh7XWig"
CLIENT_SECRET = "fKXbRxQ4GprlHPudMDnb8HKr-CHxkQ"
USER_AGENT = "python:webscrapingProject:v1.0 (by /u/Ivan_RZ)"

# Conexión en modo lectura
reddit = praw.Reddit(
    client_id=CLIENT_ID,
    client_secret=CLIENT_SECRET,
    user_agent=USER_AGENT) # Crea objeto 'reddit' para leer datos públicos en Reddit. La librería PRAW se ha descargado para lograr este acceso

# Creammos una función de nombre collect_posts. En la primera línea, le dices qué tipo de elemento esperar en subreddit_name, y qué tipo esperar en mode, y así.
def collect_posts(subreddit_name: str, mode: str = "hot", n: int = 20, time_filter: str = "day"): # Con day, le dices a la función que busque los "top" de hoy (no aplica para "hot")
    sub = reddit.subreddit(subreddit_name) # Para buscar subreddits dentro del objeto reddit
    gen = sub.hot(limit=None) if mode == "hot" else sub.top(time_filter=time_filter, limit=None) # Le pide al subreddit posteos 'hot' y 'top'.
    # Nótese que para 'hot' se usa el time_filter de día. El limit es 'None', pero se detendrá en 20 por 'n:int = 20'
    rows = [] # Crea la lista para ir metiendo cada post
    for s in gen:
        if getattr(s, "stickied", False): # stickied hace referencia a los posts pinneados (p.e. reglas de un feed). 
            continue # Si el post es pinneado, lo saltas con 'continue'. Si no está pinneado, la función lo procesa.
        rows.append({ # Agrega un diccionario con los cinco elementos que pude el ejercicio: id, title, score...
            "id": s.id,
            "title": s.title or "", # Las comillas están por si el string no tiene caracteres
            "score": int(getattr(s, "score", 0) or 0), # Usa 0 si el post de reddit no tiene score. 'int' es para que sea integer
            "num_comments": int(getattr(s, "num_comments", 0) or 0), # Misma lógica que el anterior
            "url": s.url or "", # Las comillas están por si aparece vacío el enlace
        })
        if len(rows) >= n:
            break # Termina en 20
    return rows

# Función save_csv va a recibir los rows y el path que sea creará en la computadora. Para esto último nos sirve la librería csv.
def save_csv(rows, path):
    Path(path).parent.mkdir(parents=True, exist_ok=True) # Verifica que exista la carpeta del destino, y la crea si es que no existe.
    fieldnames = ["id","title","score","num_comments","url"] # Define el orden de las columnas en csv.
    with open(path, "w", newline="", encoding="utf-8-sig") as f: # Con esto se abre el path para escribir allí archivos csv
        w = csv.DictWriter(
            f, # 'f' es objeto archivo; su nombre es arbitrario
            fieldnames=fieldnames,  # Define el orden de las columnas
            delimiter=";",             # Excel (en lenguaje ES) espera ;
            quoting=csv.QUOTE_MINIMAL  # pone comillas cuando haga falta (si el texto tiene salto de línea o comillas internas)
        )
        w.writeheader() # Escribe los encabezados
        w.writerows(rows) # Escribe todas las filas

if __name__ == "__main__": # Activa todo el código siguiente, partiendo de que estamos ejecutando directamente desde un notebook.
    subreddits = ["politics", "PoliticalDiscussion", "worldnews"] # Nombre de los foros que pide el ejercicio
    MODE = "hot" # Como la indicación dice seleccionar 20 posteos 'hot' o 'top', se entiendo que puede ser cualquiera de esas. Se está seleccionando 'hot'
    TIME_FILTER = "day"

    for sub in subreddits: # 'sub' tendrá tres valores: politics, PoliticalDiscussion y worldnews
        rows = collect_posts(sub, mode=MODE, n=20, time_filter=TIME_FILTER) # Recoge 20 post de cada subreddit
        out = ( # Construye el nombre del archivo de salida. Saldrán tres archivos porque sub tiene tres valores.
            f"data/posts_{sub}_{MODE}.csv"
            if MODE != "top" else
            f"data/posts_{sub}_{MODE}_{TIME_FILTER}.csv"
        )
        save_csv(rows, out)
        print(f"{sub}: guardados {len(rows)} posts en {out}") # Guarda cada archivo en la computadora y les pone nombre.

politics: guardados 20 posts en data/posts_politics_hot.csv
PoliticalDiscussion: guardados 20 posts en data/posts_PoliticalDiscussion_hot.csv
worldnews: guardados 20 posts en data/posts_worldnews_hot.csv


In [25]:
# Guardar todo combinado en un solo archivo csv
from pathlib import Path
import csv

all_rows = [] # Crea una lista para poner allí todos los posts de los subreddits
for sub in subreddits:
    rows_sub = collect_posts(sub, mode=MODE, n=20, time_filter=TIME_FILTER) # Llama a la función collect_posts
    for r in rows_sub: # A cada post ('r') le añadimos la columna 'subreddit', para saber de dónde viene cada post
        r["subreddit"] = sub # Define el valor de la columna
    all_rows.extend(rows_sub) # Agregar todos los elementos de 'rows_sub' a 'all_rows'

Path("data").mkdir(exist_ok=True) # Verifica que exista la carpeta del destino, y la crea si es que no existe.

with open("data/posts_combined.csv", "w", newline="", encoding="utf-8-sig") as f:
    w = csv.DictWriter(
        f,
        fieldnames=["subreddit","id","title","score","num_comments","url"],
        delimiter=";",    
        quoting=csv.QUOTE_MINIMAL
    )
    w.writeheader()
    w.writerows(all_rows) # Todo este código escribe el csv combinado

In [ ]:
# Collect Comments

In [26]:
reddit = praw.Reddit(
    client_id=CLIENT_ID,
    client_secret=CLIENT_SECRET,
    user_agent=USER_AGENT
)

# Función que toma los tres archivos (uno por subreddit) y devuelve los cinco con mayor score.
def load_top_post_ids_from_csv(csv_path, k=5):
    """Carga el CSV de posts y devuelve los IDs de los k posts con mayor score."""
    rows = [] # Crea la lista donde se guardan los csv
    with open(csv_path, newline="", encoding="utf-8-sig") as f:
        r = csv.DictReader(f, delimiter=";") # Lee cada fila del csv y arroja un diccionario por fila
        for row in r: # Recorre cada fila de la posición y convierte el score a integer
            try:
                row["score"] = int(row.get("score", 0))
            except Exception:
                row["score"] = 0
            rows.append(row) # Agrega la fila con score convertido a la lista 'rows'
    rows.sort(key=lambda x: x["score"], reverse=True) # Ordena la lista de mayor a menor en función del score de cada fila
    return [r["id"] for r in rows[:k]] # Devuelve los id de las k filas ya ordenadas

# Función que pasa los id de la función anterior, establece que se quieren cinco comentarios y el orden por score ('top')
def collect_top_comments_for_post(post_id, max_comments=5, sort="top"):
    """
    Trae hasta max_comments comentarios (en cualquier nivel), ordenados por score descendente.
    Devuelve dicts: post_id, body, score
    """
    subm = reddit.submission(id=post_id) # Crea el objeto submission ('subm') a partir de su id
    subm.comment_sort = sort # Indica el orden de los comments; ya dijimos que será según la categoría 'top'
    subm.comments.replace_more(limit=None)  # incluye también los comentarios que están en 'MoreComments' en la web

    # Lo siguiente hace que todos los comentarios estén al mismo nivel (como si todos nacieron del mismo post)
    flat = []
    for c in subm.comments.list(): # Código que efectivamente pone todos los comentarios al mismo nivel
        if not hasattr(c, "body"):
            continue # Los objetos que no tienen .body (del objeto 'comment' en la API) no son comentarios; este código hace que la función se los salte
        body = (c.body or "").strip() # Si c.body tiene texto devuelve el comentario, sino devuelve "". Luego,'strip' recorta espacios o saltos de línea. 
        if not body or body.lower() in ("[removed]", "[deleted]"):
            continue # Deje de lado aquellos comentarios que se han removido por moderación o se han borrado.
        flat.append({ # Guarda el comentarios en la lista 'flat'
            "post_id": post_id,
            "body": body.replace("\n", " "),
            "score": int(getattr(c, "score", 0) or 0),
        })
    flat.sort(key=lambda x: x["score"], reverse=True)
    return flat[:max_comments] # Te arroja los cinco comentarios de mayor score del hilo completo

# Función para abrir el path y guardar allí los archivos.
def save_comments_csv(rows, path):
    Path(path).parent.mkdir(parents=True, exist_ok=True)
    with open(path, "w", newline="", encoding="utf-8-sig") as f:
        w = csv.DictWriter(
            f,
            fieldnames=["post_id", "body", "score"],
            delimiter=";",
            quoting=csv.QUOTE_MINIMAL,
        )
        w.writeheader()
        w.writerows(rows)

        
if __name__ == "__main__": # Activa todo el código siguiente, partiendo de que estamos ejecutando directamente desde un notebook.
    subreddits = ["politics", "PoliticalDiscussion", "worldnews"] # Nombre de los foros que pide el ejercicio
    MODE = "hot" # Como la indicación dice seleccionar 20 posteos 'hot' o 'top', se entiendo que puede ser cualquiera de esas. Se está seleccionando 'hot'       
    TIME_FILTER = "day"  # solo aplica si MODE == "top"

    for sub in subreddits: # 'sub' tendrá tres valores: politics, PoliticalDiscussion y worldnews
        posts_csv = (
            f"data/posts_{sub}_{MODE}.csv"
            if MODE != "top" else
            f"data/posts_{sub}_{MODE}_{TIME_FILTER}.csv") # Le pone nombre a los archivos, en función del mode ('top' o 'hot')

        # Elegimos el conjunto de posts más relevantes (top 5 por score), del objeto 'posts_csv'
        top_post_ids = load_top_post_ids_from_csv(posts_csv, k=5)

        # Esto hace que nos quedemos con 5 comentarios por post
        all_comments = []
        for pid in top_post_ids: # Recorre los id de los cincos mejores posts
            comments = collect_top_comments_for_post(pid, max_comments=5, sort="top") # Pide hasta cinco comentarios
            all_comments.extend(comments) # Agrega los comentarios

        # Guardar comentarios en tres archivos csv (uno por subreddit)
        out_csv = f"data/comments_{sub}.csv"
        save_comments_csv(all_comments, out_csv)
        print(f"{sub}: guardados {len(all_comments)} comentarios en {out_csv}")


politics: guardados 25 comentarios en data/comments_politics.csv
PoliticalDiscussion: guardados 25 comentarios en data/comments_PoliticalDiscussion.csv
worldnews: guardados 25 comentarios en data/comments_worldnews.csv


In [ ]:
# Storage

In [27]:
# Emparentar comentarios con sus posteos madre
import csv
from pathlib import Path

# Función que creará un csv para que cada comentario quede enlazado a su post madre
def build_linked_comments_csv(subreddits, mode="hot", time_filter="day",
                              out_path="data/comments_linked.csv"):
    post_index = {}  # Asocia el id de cada post con un diccionario (subreddit:..., title:...)
    for sub in subreddits: # Recorre la lista de subreddits
        posts_csv = ( # Construye el nombre de archivos que ya se han creado en el primer chunk
            f"data/posts_{sub}_{mode}.csv"
            if mode != "top" else
            f"data/posts_{sub}_{mode}_{time_filter}.csv"
        )
        with open(posts_csv, newline="", encoding="utf-8-sig") as f: # Lee archivos post_csv
            r = csv.DictReader(f, delimiter=";")
            for row in r:
                pid = row["id"]
                post_index[pid] = {
                    "subreddit": sub,
                    "title": row.get("title", ""),
                    "url": row.get("url", ""),
                }
    linked_rows = [] # Acá se guardarán las filas combinadas (comentarios + otros datos)
    orphans = 0
    for sub in subreddits: # Recorre cada subreddit y arma el path del csv de comentarios
        comments_csv = f"data/comments_{sub}.csv"
        if not Path(comments_csv).exists(): # Si el archivo no existe, lo salta
            continue
        with open(comments_csv, newline="", encoding="utf-8-sig") as f: # Abre el csv de comentarios
            r = csv.DictReader(f, delimiter=";")
            for c in r: # Itera cada comentario y lo relaciona con el post madre (por medio de 'post_index')
                pid = c["post_id"]
                info = post_index.get(pid)
                if not info:
                    orphans += 1  # comentario cuyo post no encontramos
                    continue
                #Construye la fila combinada y lo agrega a 'linked_rows'
                linked_rows.append({
                    "subreddit": info["subreddit"],
                    "post_id": pid,
                    "post_title": info["title"],
                    "post_url": info["url"],
                    "comment_body": c.get("body", ""),
                    "comment_score": c.get("score", 0),
                })

    # Escribe y guarda archivo csv combinado 
    Path(out_path).parent.mkdir(parents=True, exist_ok=True)
    with open(out_path, "w", newline="", encoding="utf-8-sig") as f:
        w = csv.DictWriter(
            f,
            fieldnames=["subreddit","post_id","post_title","post_url","comment_body","comment_score"],
            delimiter=";",
            quoting=csv.QUOTE_MINIMAL
        )
        w.writeheader()
        w.writerows(linked_rows)

    print(f"Comentarios linkeados guardados en: {out_path} (filas: {len(linked_rows)}, huérfanos omitidos: {orphans})")

# Ejecuta la función, definiendo los parámetros de la corrida
subreddits = ["politics", "PoliticalDiscussion", "worldnews"]
MODE = "hot"         # también podría set "top"
TIME_FILTER = "day"  # solo si MODE == "top"
build_linked_comments_csv(subreddits, mode=MODE, time_filter=TIME_FILTER)


Comentarios linkeados guardados en: data/comments_linked.csv (filas: 75, huérfanos omitidos: 0)
